In [1]:
from system import *
from math import floor
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as color

plt.rcParams.update({'font.size': 18})

e_field = zero_field()
b_field = t89(1)
system  = System(e_field, b_field)

Load IGRF coefficients ...


In [200]:
degs = []
min_pas = []
max_pas = []

In [348]:
deg = 89.99

system.populate_by_eq_pa(500,
                         delta(12),
                         delta(2e6),
                         delta(np.radians(deg)),
                         uniform(0, 2*np.pi)
                        )

100%|██████████| 500/500 [00:02<00:00, 242.05it/s]


In [349]:
system.solve(1, 1e-4)

100%|██████████| 500/500 [00:45<00:00, 11.02it/s]


In [350]:
pas = pitch_angle(system.history)
v   = velocity(system.history)
rr  = position(system.history)
b   = b_mag(system.history)
mom = eq_pitch_angle_from_moment(system.history, system.ics[:, 4, 0:2])

adp_grid = np.zeros((len(system.history[:, 0, 0, 0]), len(system.history[0, :, 0, 0])))
pa_grid  = np.zeros((len(system.history[:, 0, 0, 0]), len(system.history[0, :, 0, 0])))

for i in range(len(rr[:, 0])):
    for j in range(len(rr[0, :])):
        rho_o          = gamma(v[i, j]) * sp.m_e * np.linalg.norm(v[i, j]) / (abs(-sp.e) * b[i, j])
        R_c            = flc(b_field, rr[i, j])
        adp_grid[i, j] = rho_o / R_c
        pa_grid[i, j]  = pas[i, j]

In [351]:
scattered_pas = np.zeros(len(system.history[:, 0, 0, 0]))

for i in range(len(system.history[:, 0, 0, 0])):
    near_mirror = np.argwhere(np.abs(pa_grid[i, :] - 90) <= 1)[:, 0]
    ind = np.argwhere(adp_grid[i, near_mirror] < 0.1)[:, 0]
    if len(near_mirror[ind]) < 2:
        scattered_pas[i] = 0
        continue
    elif near_mirror[ind][0] > 10:
        scattered_pas[i] = (0.5*np.sin(np.radians(mom[i, near_mirror[ind][0]]))**2) / (0.5 * np.sin(np.radians(deg))**2)
    else:
        scattered_pas[i] = (0.5*np.sin(np.radians(mom[i, near_mirror[ind][1]]))**2) / (0.5 * np.sin(np.radians(deg))**2)

In [352]:
spas = scattered_pas[np.where(scattered_pas != 0)]
min_pa, max_pa = (np.amin(spas), np.amax(spas))

In [353]:
degs.append(deg)
min_pas.append(min_pa)
max_pas.append(max_pa)

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

smooth = 17 # Original: 17

spl_min = make_interp_spline(degs[:], min_pas[:], k=3)
power_smooth_min = spl_min(np.linspace(degs[0], degs[-1], smooth))
spl_max = make_interp_spline(degs[:], max_pas[:], k=3)
power_smooth_max = spl_max(np.linspace(degs[0], degs[-1], smooth))

plt.figure(figsize=(10, 8))
plt.plot(np.linspace(degs[0], degs[-1], smooth), power_smooth_min, color='m', label=r'$\mu_{min}$')
plt.plot(np.linspace(degs[0], degs[-1], smooth), power_smooth_max, color='r', label=r'$\mu_{max}$')
plt.legend()
plt.xlim([10, 90])
plt.ylim(1e-2, 2)
plt.yscale('log')
plt.ylabel(r'$\mu / \mu_0$')
plt.xlabel(r'Initial $\alpha_0$ (deg)')
plt.title('Final vs. initial $\mu$ (2 MeV e$^-$)')
plt.grid(which='both')
plt.show()